In [52]:
from math import *
import numpy as np
from PIL import Image

In [89]:
image = Image.open(r'canvas.png')

ImageW, ImageH = image.size
dist = 10
WindowW = 10
WindowH = 10
BACKGROUND_COLOR = np.array([0, 0, 0])

class sphere():
    def __init__(self, ce, r, co, spe):
        self.center = ce
        self.radius = r
        self.color = co
        self.specular = spe

spheres = []

spheres.append(sphere(np.array([0.8, 0.78, 3]), 0.2, np.array([0, 255, 255]), 500))
spheres.append(sphere(np.array([0.2, 0.2, 5]), 0.8, np.array([255, 255, 255]), 10))
spheres.append(sphere(np.array([-0.4, 0, 15]), 3, np.array([230, 255, 24]), 1000))

def CanvasToView(x,y):
    return np.array([x*WindowW/ImageW, -y*WindowH/ImageH, dist])

def IntersectRaySphere(O, d, sphere):
    center = sphere.center
    radius = sphere.radius
    oc = O - center

    k1 = np.sum(np.dot(d, d))
    k2 = 2*np.sum(np.dot(oc, d))
    k3 = np.sum(np.dot(oc, oc)) - radius*radius

    discriminant = k2*k2 - 4*k1*k3
    if discriminant < 0:
        return inf, inf

    t1 = (-k2 + sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - sqrt(discriminant)) / (2*k1)
    return t1, t2

def TraceRay(O, d, tmin, tmax):
    closest_sphere, closest_t = ClosestIntersection(O, d, tmin, tmax)
        
    if closest_sphere == None:
        return BACKGROUND_COLOR
    
    P = O + np.dot(closest_t, d)  # вычисление пересечения
    N = P - closest_sphere.center  # вычисление нормали сферы в точке пересечения
    N = N / sqrt(np.sum(np.square(N)))
    return closest_sphere.color * Lighting(P, N, -d, closest_sphere.specular)

In [90]:
def ClosestIntersection(O, d, tmin, tmax):
    closest_t = inf
    closest_sphere = None
    
    for sphere in spheres:
        t1, t2 = IntersectRaySphere(O, d, sphere)
        if t1 >= tmin and t1 <= tmax and t1 < closest_t:
            closest_t = t1
            closest_sphere = sphere
        if t2 >= tmin and t2 <= tmax and t2 < closest_t:
            closest_t = t2
            closest_sphere = sphere
    return closest_sphere, closest_t


In [91]:
#освещение

class light1:
    typelight = 'ambient'
    intensity = 0.2

class light2:
    typelight = 'point'
    intensity = 0.6
    position = np.array([2, 1, 0])

"""
class light3:
    typelight = 'directional'
    intensity = 0.2
    direction = np.array([1, 4, 4])
    
"""

lights = []
lights.append(light1)
lights.append(light2)
#lights.append(light3)

def Lighting(P, N, V, s):
    i = 0.0
    for light in lights:
        if light.typelight == 'ambient':
            i += light.intensity
        else:
            if light.typelight == 'point':
                L = light.position - P
                tmax = 1
            #else:
                #L = light.direction
                #tmax = inf
                
            # Проверка тени
            shadow_sphere, shadow_t = ClosestIntersection(P, L, 0.001, tmax)
            if shadow_sphere != None:
                continue
                
            # матовые
            n_dot_l = np.sum(np.dot(N, L))
            if n_dot_l > 0:
                i += light.intensity*n_dot_l/(sqrt(np.sum(np.square(N))) * sqrt(np.sum(np.square(L))))
                
            # зеркальные
            if s != -1:
                R = 2 * N * np.sum(np.dot(N, L)) - L
                r_dot_v = np.sum(np.dot(R, V))
                if r_dot_v > 0:
                    i += light.intensity * pow(r_dot_v/(sqrt(np.sum(np.square(R)))*sqrt(np.sum(np.square(V)))), s)
    return i


In [92]:
O = np.array([0,0,0])
for x in range(-300, 300):
    xx = x + 300
    for y in range(-300, 300):
        yy = y + 300
        color = TraceRay(O, CanvasToView(x, y), 0, inf)
        image.putpixel((xx, yy), tuple(int(c) for c in color))
image.show()